In [3]:
# ---------------------------------------------
# Facial Expression Recognition - Optimized Training
# Notebook 2 (Optimized): Lightweight CNN + Callbacks + Class Weights
# ---------------------------------------------

# 1. Imports
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import numpy as np
import pickle
import os
from datetime import datetime

# 2. Load preprocessed data
print("Loading datasets...")
X_train = np.load('../data/X_train.npy')
X_val = np.load('../data/X_val.npy')
y_train = np.load('../data/y_train.npy')  # One-hot encoded
y_val = np.load('../data/y_val.npy')

with open('../data/class_weight.pkl', 'rb') as f:
    class_weight_dict = pickle.load(f)

with open('../data/class_names.pkl', 'rb') as f:
    class_names = pickle.load(f)

print("Train set:", X_train.shape)
print("Validation set:", X_val.shape)
print("Number of classes:", len(class_names))

# 3. Optimized CNN Model
def build_optimized_model(input_shape=(48, 48, 1), num_classes=7):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.2),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.3),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2, 2),
        layers.Dropout(0.4),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = build_optimized_model(input_shape=(48, 48, 1), num_classes=len(class_names))

# 4. Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 5. Callbacks
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = f'../models/expression_model_{timestamp}.h5'

callbacks_list = [
    callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
]

# 6. Train
print("\nTraining optimized model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=callbacks_list,
    verbose=1
)

# 7. Save training history
with open(f'../models/training_history_{timestamp}.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print(f"\nModel saved to: {model_path}")
print(f"Training history saved to: ../models/training_history_{timestamp}.pkl")

# 8. Final Accuracy Report
print(f"\nFinal Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Val Accuracy: {history.history['val_accuracy'][-1]:.4f}")

Loading datasets...
Train set: (25838, 48, 48, 1)
Validation set: (2871, 48, 48, 1)
Number of classes: 7
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_4 (Bat  (None, 48, 48, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 24, 24, 32)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 24, 24, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 64)        18496     
               

C:\Users\rohit\anaconda3\envs\myEnv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


404/404 [==============================] - 315s 755ms/step - loss: 2.1345 - accuracy: 0.2437 - val_loss: 2.6518 - val_accuracy: 0.1818 - lr: 0.0010
Epoch 2/30
404/404 [==============================] - ETA: 0s - loss: 1.7050 - accuracy: 0.3412  
Epoch 2: val_accuracy improved from 0.18182 to 0.38070, saving model to ../models\expression_model_20250509-093652.h5
404/404 [==============================] - 301s 746ms/step - loss: 1.7050 - accuracy: 0.3412 - val_loss: 1.6096 - val_accuracy: 0.3807 - lr: 0.0010
Epoch 3/30
404/404 [==============================] - ETA: 0s - loss: 1.5571 - accuracy: 0.3906  
Epoch 3: val_accuracy improved from 0.38070 to 0.38488, saving model to ../models\expression_model_20250509-093652.h5
404/404 [==============================] - 304s 752ms/step - loss: 1.5571 - accuracy: 0.3906 - val_loss: 1.6694 - val_accuracy: 0.3849 - lr: 0.0010
Epoch 4/30
404/404 [==============================] - ETA: 0s - loss: 1.4783 - accuracy: 0.4231  
Epoch 4: val_accuracy did 